In [7]:
!pip install tabula-py pandas jpype1 -q

In [10]:
import os
os.environ["JAVA_HOME"] = "/opt/homebrew/opt/openjdk/libexec/openjdk.jdk/Contents/Home"
os.environ["PATH"] += os.pathsep + os.path.join(os.environ["JAVA_HOME"], "bin")

import tabula, pandas as pd
print("Java Path:", os.environ["JAVA_HOME"])

Java Path: /opt/homebrew/opt/openjdk/libexec/openjdk.jdk/Contents/Home


In [11]:
import re
import pandas as pd
import tabula
from pathlib import Path

# ---------------- config ----------------
EXPECTED_COLS = ["ลำดับ","ชื่อสามัญ","ชื่อพื้นเมือง","ชื่อวิทยาศาสตร์แบบเต็ม","ชื่อวิทยาศาสตร์","วงศ์"]

HEADER_HINTS = [
    "ลำดับ","ชื่อสามัญ","ชื่อพื้นเมือง","ชื่อวิทยาศาสตร์แบบเต็ม","ชื่อวิทยาศาสตร์","ชื่อคนค้นพบ","วงศ์"
]

# เผื่อ OCR แตกคำ/สะกดเพี้ยนเล็กน้อย ใส่ synoynms ที่มักเจอบ่อยได้ที่นี่
HEADER_SYNONYMS = {
    "ชื่อวิทยาศาสตร์แบบเต็ม": ["ชื่อวิทยาศาสตร์แบบเต็ม", "ชื่อวิทยาศาสตร์แบบ เต็ม", "ชื่อวิทย์แบบเต็ม"],
    "ชื่อวิทยาศาสตร์": ["ชื่อวิทยาศาสตร์","ชื่อวิทย์","ชื่อวิทยาศาสตร์ฯ"],
}

def looks_like_header_row(row_vals):
    text = " ".join([str(x) for x in row_vals])
    return any(h in text for h in HEADER_HINTS)

def promote_first_row_as_header(df: pd.DataFrame) -> pd.DataFrame:
    """กรณีที่หัวคอลัมน์หล่นไปอยู่แถวแรก ให้ยกขึ้นเป็น header"""
    if df.empty: 
        return df
    first = [str(x).strip() for x in df.iloc[0].tolist()]
    if looks_like_header_row(first):
        df2 = df.copy()
        df2.columns = first
        df2 = df2.iloc[1:].reset_index(drop=True)
        return df2
    return df

def normalize_headers(df: pd.DataFrame) -> pd.DataFrame:
    """เคลียร์ชื่อหัวคอลัมน์ให้สะอาด และ map synonyms"""
    # ตัดช่องว่างซ้ำ
    new_cols = []
    for c in df.columns:
        c2 = re.sub(r"\s+", " ", str(c)).strip()
        new_cols.append(c2)
    df.columns = new_cols
    
    # map synonyms เฉพาะคอลัมน์วิทยาศาสตร์
    col_map = {}
    for col in df.columns:
        target = None
        for k, alts in HEADER_SYNONYMS.items():
            if col == k or any(col == a for a in alts):
                target = k
                break
        col_map[col] = target if target else col
    return df.rename(columns=col_map)

def standardize_to_expected(df: pd.DataFrame) -> pd.DataFrame:
    """เลือกและเรียงคอลัมน์ตาม EXPECTED_COLS; ไม่พบให้เติมค่าว่าง"""
    out = pd.DataFrame()
    for c in EXPECTED_COLS:
        if c in df.columns:
            out[c] = df[c]
        else:
            out[c] = pd.NA
    return out

def clean_numeric_index(s):
    """พยายามแปลงคอลัมน์ 'ลำดับ' ให้เป็นตัวเลข (ถอด , และช่องว่าง)"""
    if pd.isna(s): 
        return pd.NA
    t = re.sub(r"[^\d]", "", str(s))
    return int(t) if t.isdigit() else pd.NA

def read_all_tables(pdf_path: str) -> pd.DataFrame:
    # ดึงทุกตารางทุกหน้า (lattice สำหรับไฟล์มีเส้นกรอบ)
    dfs = tabula.read_pdf(pdf_path, pages="all", multiple_tables=True, lattice=True)
    cleaned = []

    for d in dfs:
        if d is None or d.empty:
            continue
        d = promote_first_row_as_header(d)
        d = normalize_headers(d)

        # กรองแถวหัวซ้ำๆ ที่บางที Tabula ดึงมาเป็นข้อมูลอีก
        if not d.empty and looks_like_header_row(d.columns.tolist()):
            # หมายความว่าชื่อคอลัมน์ยังเป็นค่า non-sense → มักเป็นกรณี OCR เพี้ยน
            d = promote_first_row_as_header(d)

        # บางหน้าอาจแยกเป็นสองตาราง (บน/ล่าง) — เราเก็บหมด ไปล้างรวมทีหลัง
        cleaned.append(d)

    if not cleaned:
        return pd.DataFrame(columns=EXPECTED_COLS)

    big = pd.concat(cleaned, ignore_index=True)

    # ตัดคอลัมน์ที่ Tabula แปลกๆ เช่น "Unnamed: 0"
    big = big.loc[:, ~big.columns.str.contains("^Unnamed")]

    # เลือกเฉพาะคอลัมน์ที่เกี่ยวข้อง แล้วจัดเรียงให้ตรงตามที่ต้องการ
    big_std = standardize_to_expected(big)

    # ทำความสะอาดค่า
    # ลำดับ → ตัวเลข
    big_std["ลำดับ"] = big_std["ลำดับ"].map(clean_numeric_index)
    big_std = big_std.dropna(subset=["ลำดับ"]).copy()
    big_std["ลำดับ"] = big_std["ลำดับ"].astype("Int64")

    # ตัดช่องว่างซ้ำทุกคอลัมน์ข้อความ
    for c in ["ชื่อสามัญ","ชื่อพื้นเมือง","ชื่อวิทยาศาสตร์แบบเต็ม","ชื่อวิทยาศาสตร์","วงศ์"]:
        big_std[c] = big_std[c].astype("string").str.replace(r"\s+", " ", regex=True).str.strip()

    # กรองหัวตารางที่ยังเล็ดรอดเป็นแถวข้อมูล (แถวที่ 'วงศ์' เท่ากับ 'วงศ์' เป็นต้น)
    mask_header_like = (
        (big_std["ชื่อสามัญ"].fillna("") == "ชื่อสามัญ") |
        (big_std["ชื่อวิทยาศาสตร์"].fillna("") == "ชื่อวิทยาศาสตร์") |
        (big_std["วงศ์"].fillna("") == "วงศ์")
    )
    big_std = big_std.loc[~mask_header_like].reset_index(drop=True)

    return big_std

In [12]:
pdf_path = "รายชื่อพันธุ์ไม้สามัญและวิทยาศาสตร์2022.pdf"  # เปลี่ยนเป็นพาธไฟล์ของคุณ
df = read_all_tables(pdf_path)
print(df.shape)
df.head(12)

: 

In [ ]:
df.to_csv("trees_parsed.csv", index=False, encoding="utf-8-sig")
"saved: trees_parsed.csv"